# <center>Generators</center>
## <center>Practical cases</center>

### Table of contents
---
- **Cases.**
- **Pipeline (Unix).**
- **Building a real-world _Shell_ command into script.**
- **Benchmarking.**
---

In [ ]:
# Example of using shell commands
# %%bash